## Step 70: Export SWC PREC Filesets

Purpose: Export SWC PREC datasets from the **results** file geodatabase as tab-delimited files


In [1]:
import arcpy;
import os,sys;
import json,requests;
import datetime,http.client;
import zipfile,shutil,csv;
import time;

print("Step 70: Export SWC PREC Filesets");

def swc_resources():
    
    rez = {};
    
    # Verify or Create Source filegeodatabase
    rez['source'] = os.getcwd() + os.sep + 'source.gdb';

    if not arcpy.Exists(rez['source']):
        print("  creating new source workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['source'])
            ,os.path.basename(rez['source'])
        );
        
    # Verify or Create Working filegeodatabase
    rez['working'] = os.getcwd() + os.sep + 'working.gdb';

    if not arcpy.Exists(rez['working']):
        print("  creating new working workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['working'])
            ,os.path.basename(rez['working'])
        );
        
    # Verify or Create Results filegeodatabase
    rez['results'] = os.getcwd() + os.sep + 'results.gdb';

    if not arcpy.Exists(rez['results']):
        print("  creating new results workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['results'])
            ,os.path.basename(rez['results'])
        );

    # Verify or Create qa directory
    rez['qa'] = os.getcwd() + os.sep + 'qa';

    if not arcpy.Exists(rez['qa']):
        print("  creating new qa directory");
        os.mkdir(rez['qa']);
        
    # Verify or Create resources directory
    rez['resources'] = os.getcwd() + os.sep + 'resources';

    if not arcpy.Exists(rez['resources']):
        print("  creating new resources directory");
        os.mkdir(rez['resources']);
        
    # Verify existence of files directory
    rez['files'] = os.getcwd() + os.sep + 'files';
    
    if not arcpy.Exists(rez['resources']):
        raise Exception('ERROR: project files directory not found');
        
    return rez;

rez = swc_resources();

station_cnt                = arcpy.GetCount_management(rez['results'] + os.sep + 'D4EMLite_PREC_Details')[0];

PREC2035HotDry_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2035HotDry')[0];
PREC2035Central_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2035Central')[0];
PREC2035WetWarm_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2035WetWarm')[0];

PREC2060HotDry_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2060HotDry')[0];
PREC2060Central_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2060Central')[0];
PREC2060WetWarm_cnt        = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC2060WetWarm')[0];

GEVdepth2035Stormy_cnt     = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2035Stormy')[0];
GEVdepth2035LessStormy_cnt = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2035LessStormy')[0];

GEVdepth2060Stormy_cnt     = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2060Stormy')[0];
GEVdepth2060LessStormy_cnt = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth2060LessStormy')[0];

GEVdepth_historical_cnt    = arcpy.GetCount_management(rez['results'] + os.sep + 'GEVdepth_historical')[0];

PREC_SCS_Types_cnt         = arcpy.GetCount_management(rez['results'] + os.sep + 'PREC_SCS_Types')[0];

print("  Station Count               : " + str(station_cnt));

print("  PREC2035HotDry Count        : " + str(PREC2035HotDry_cnt));
print("  PREC2035Central Count       : " + str(PREC2035Central_cnt));
print("  PREC2035WetWarm Count       : " + str(PREC2035WetWarm_cnt));

print("  PREC2060HotDry Count        : " + str(PREC2060HotDry_cnt));
print("  PREC2060Central Count       : " + str(PREC2060Central_cnt));
print("  PREC2060WetWarm Count       : " + str(PREC2060WetWarm_cnt));

print("  GEVdepth2035Stormy Count    : " + str(GEVdepth2035Stormy_cnt));
print("  GEVdepth2035LessStormy Count: " + str(GEVdepth2035LessStormy_cnt));

print("  GEVdepth2060Stormy Count    : " + str(GEVdepth2060Stormy_cnt));
print("  GEVdepth2060LessStormy Count: " + str(GEVdepth2060LessStormy_cnt));

print("  GEVdepth_historical Count   : " + str(GEVdepth_historical_cnt));

print("  PREC_SCS_Types Count        : " + str(PREC_SCS_Types_cnt));


Step 70: Export SWC PREC Filesets
  Station Count               : 5144
  PREC2035HotDry Count        : 5144
  PREC2035Central Count       : 5144
  PREC2035WetWarm Count       : 5144
  PREC2060HotDry Count        : 5144
  PREC2060Central Count       : 5144
  PREC2060WetWarm Count       : 5144
  GEVdepth2035Stormy Count    : 5144
  GEVdepth2035LessStormy Count: 5144
  GEVdepth2060Stormy Count    : 5144
  GEVdepth2060LessStormy Count: 5144
  GEVdepth_historical Count   : 5144
  PREC_SCS_Types Count        : 5144


In [2]:
def tabWriter(fc,outfile,flds,outflds=None):

    if outflds is None:
        outflds = flds;
        
    with open(outfile,'w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f,delimiter='\t',lineterminator='\n');

        with arcpy.da.SearchCursor(fc,flds) as incur:
            writer.writerow(outflds);

            for row in incur:
                if row[0] is not None:
                    writer.writerow(row);

    cnt = arcpy.GetCount_management(outfile)[0];         
    print("  wrote " + str(cnt) + " records to " + os.path.basename(outfile) + ".");


### 70.010: Generate the D4EMLite_PREC_Details tab-delimited text file of stations

In [3]:
%%time

if arcpy.Exists(rez['resources'] + os.sep + 'D4EMLite_PREC_Details.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'D4EMLite_PREC_Details');
  
flds = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
];

z = tabWriter(
     fc      = rez['results'] + os.sep + 'D4EMLite_PREC_Details'
    ,outfile = rez['resources'] + os.sep + 'D4EMLite_PREC_Details.txt'
    ,flds    = flds
);


  wrote 5144 records to D4EMLite_PREC_Details.txt.
Wall time: 739 ms


### 70.020: Generate TEMP and PREC tab-delimited text files

In [4]:
%%time

if arcpy.Exists(rez['resources'] + os.sep + 'TEMP2035HotDry.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'TEMP2035HotDry.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'TEMP2035Central.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'TEMP2035Central.txt');
    
if arcpy.Exists(rez['resources'] + os.sep + 'TEMP2035WetWarm.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'TEMP2035WetWarm.txt');
    
if arcpy.Exists(rez['resources'] + os.sep + 'TEMP2060HotDry.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'TEMP2060HotDry.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'TEMP2060Central.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'TEMP2060Central.txt');
    
if arcpy.Exists(rez['resources'] + os.sep + 'TEMP2060WetWarm.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'TEMP2060WetWarm.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'PREC2035HotDry.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'PREC2035HotDry.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'PREC2035Central.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'PREC2035Central.txt');
    
if arcpy.Exists(rez['resources'] + os.sep + 'PREC2035WetWarm.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'PREC2035WetWarm.txt');
    
if arcpy.Exists(rez['resources'] + os.sep + 'PREC2060HotDry.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'PREC2060HotDry.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'PREC2060Central.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'PREC2060Central.txt');
    
if arcpy.Exists(rez['resources'] + os.sep + 'PREC2060WetWarm.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'PREC2060WetWarm.txt');
    
flds = [
     'StationId'
    ,'Jan'
    ,'Feb'
    ,'Mar'
    ,'Apr'
    ,'May'
    ,'Jun'
    ,'Jul'
    ,'Aug'
    ,'Sep'
    ,'Oct'
    ,'Nov'
    ,'Dec'
    ,'Ann'
];

z = tabWriter(
     fc      = rez['results'] + os.sep + 'TEMP2035HotDry'
    ,outfile = rez['resources'] + os.sep + 'TEMP2035HotDry.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'TEMP2035Central'
    ,outfile = rez['resources'] + os.sep + 'TEMP2035Central.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'TEMP2035WetWarm'
    ,outfile = rez['resources'] + os.sep + 'TEMP2035WetWarm.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'TEMP2060HotDry'
    ,outfile = rez['resources'] + os.sep + 'TEMP2060HotDry.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'TEMP2060Central'
    ,outfile = rez['resources'] + os.sep + 'TEMP2060Central.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'TEMP2060WetWarm'
    ,outfile = rez['resources'] + os.sep + 'TEMP2060WetWarm.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC2035HotDry'
    ,outfile = rez['resources'] + os.sep + 'PREC2035HotDry.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC2035Central'
    ,outfile = rez['resources'] + os.sep + 'PREC2035Central.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC2035WetWarm'
    ,outfile = rez['resources'] + os.sep + 'PREC2035WetWarm.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC2060HotDry'
    ,outfile = rez['resources'] + os.sep + 'PREC2060HotDry.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC2060Central'
    ,outfile = rez['resources'] + os.sep + 'PREC2060Central.txt'
    ,flds    = flds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC2060WetWarm'
    ,outfile = rez['resources'] + os.sep + 'PREC2060WetWarm.txt'
    ,flds    = flds
);


  wrote 5144 records to TEMP2035HotDry.txt.
  wrote 5144 records to TEMP2035Central.txt.
  wrote 5144 records to TEMP2035WetWarm.txt.
  wrote 5144 records to TEMP2060HotDry.txt.
  wrote 5144 records to TEMP2060Central.txt.
  wrote 5144 records to TEMP2060WetWarm.txt.
  wrote 5144 records to PREC2035HotDry.txt.
  wrote 5144 records to PREC2035Central.txt.
  wrote 5144 records to PREC2035WetWarm.txt.
  wrote 5144 records to PREC2060HotDry.txt.
  wrote 5144 records to PREC2060Central.txt.
  wrote 5144 records to PREC2060WetWarm.txt.
Wall time: 2.76 s


### 70.030: Generate GEVdepth tab-delimited files

In [5]:
%%time

if arcpy.Exists(rez['resources'] + os.sep + 'GEVdepth2035Stormy.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'GEVdepth2035Stormy.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'GEVdepth2035LessStormy.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'GEVdepth2035LessStormy.txt'); 

if arcpy.Exists(rez['resources'] + os.sep + 'GEVdepth2060Stormy.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'GEVdepth2060Stormy.txt');

if arcpy.Exists(rez['resources'] + os.sep + 'GEVdepth2060LessStormy.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'GEVdepth2060LessStormy.txt'); 

flds = [
     'StationId'
    ,'z5yrann_mm'
    ,'z10yrann_mm'
    ,'z15yrann_mm'
    ,'z30yrann_mm'
    ,'z50yrann_mm'
    ,'z100yrann_mm'
];

outflds = [
     'StationId'
    ,'5yrann_mm'
    ,'10yrann_mm'
    ,'15yrann_mm'
    ,'30yrann_mm'
    ,'50yrann_mm'
    ,'100yrann_mm'
];

z = tabWriter(
     fc      = rez['results'] + os.sep + 'GEVdepth2035Stormy'
    ,outfile = rez['resources'] + os.sep + 'GEVdepth2035Stormy.txt'
    ,flds    = flds
    ,outflds = outflds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'GEVdepth2035LessStormy'
    ,outfile = rez['resources'] + os.sep + 'GEVdepth2035LessStormy.txt'
    ,flds    = flds
    ,outflds = outflds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'GEVdepth2060Stormy'
    ,outfile = rez['resources'] + os.sep + 'GEVdepth2060Stormy.txt'
    ,flds    = flds
    ,outflds = outflds
);

z = tabWriter(
     fc      = rez['results']+ os.sep + 'GEVdepth2060LessStormy'
    ,outfile = rez['resources'] + os.sep + 'GEVdepth2060LessStormy.txt'
    ,flds    = flds
    ,outflds = outflds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'GEVdepth_historical'
    ,outfile = rez['resources'] + os.sep + 'GEVdepth_historical.txt'
    ,flds    = flds
    ,outflds = outflds
);

z = tabWriter(
     fc      = rez['results'] + os.sep + 'PREC_SCS_Types'
    ,outfile = rez['resources'] + os.sep + 'PREC_SCS_Types.txt'
    ,flds    = ['StationId','Rainfall_Distribution_Type']
);


  wrote 5144 records to GEVdepth2035Stormy.txt.
  wrote 5144 records to GEVdepth2035LessStormy.txt.
  wrote 5144 records to GEVdepth2060Stormy.txt.
  wrote 5144 records to GEVdepth2060LessStormy.txt.
  wrote 5144 records to GEVdepth_historical.txt.
  wrote 5144 records to PREC_SCS_Types.txt.
Wall time: 1.19 s


### 70.040: Generate new SCS24Hour.txt reference file

In [6]:
%%time

if arcpy.Exists(rez['resources'] + os.sep + 'SCS24Hour.txt'):
    arcpy.Delete_management(rez['resources'] + os.sep + 'SCS24Hour.txt');

print("  writing new SCS24Hour.txt file");
flds = [
     'Time'
    ,'CA_1'
    ,'CA_2'
    ,'CA_3'
    ,'CA_4'
    ,'CA_5'
    ,'CA_6'
    ,'MSE_1'
    ,'MSE_2'
    ,'MSE_3'
    ,'MSE_4'
    ,'MSE_5'
    ,'MSE_6'
    ,'NOAA_A'
    ,'NOAA_B'
    ,'NOAA_C'
    ,'NOAA_D'
    ,'NRCC_A'
    ,'NRCC_B'
    ,'NRCC_C'
    ,'NRCC_D'
    ,'NV_N'
    ,'NV_S'
    ,'NV_W'
    ,'SCS_I'
    ,'SCS_IA'
    ,'SCS_II'
    ,'SCS_III'
];

z = tabWriter(
     fc      = rez['source'] + os.sep + 'TEMPORAL_DIST_FILE'
    ,outfile = rez['resources'] + os.sep + 'SCS24Hour.txt'
    ,flds    = flds
);


  writing new SCS24Hour.txt file
  wrote 241 records to SCS24Hour.txt.
Wall time: 197 ms
